In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import ta
from fucnt_opt import TradingStrategyOptimizer
from fucnt_opt import clean_ds
from fucnt_opt import dnn_strategy




/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-07 16:14:11.535829: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 16:14:11.536118: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 16:14:11.538521: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 16:14:11.564834: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instru

In [2]:
optimizer = TradingStrategyOptimizer('/home/rodo/code/proyecto_4/004_DL/data/close_data_buy_5.csv', '/home/rodo/code/proyecto_4/004_DL/data/close_data_sell_5.csv')
df = optimizer.run()
df

[I 2024-04-07 16:14:12,831] A new study created in memory with name: no-name-1d03b6c3-d6fe-41e9-8a72-29fbd8887c20
/home/rodo/code/proyecto_4/004_DL/deep_learning/fucnt_opt.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-04-07 16:14:13.476027: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been b

1236/1236 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 1s 564us/step


,Y_BUY_PRED,Y_SELL_PRED
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
39533,True,False
39534,True,False
39535,True,False
39536,True,False


In [3]:


df1 = pd.read_csv("../data/aapl_5m_train.csv")
df_5min = clean_ds(df1)
close_data = df_5min[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']]
close_data = close_data.dropna()

df_indexed = df.reset_index()
close_data_indexed = close_data.reset_index()

close_data_updated = close_data_indexed.join(df_indexed[['Y_BUY_PRED', 'Y_SELL_PRED']])



closes_5min = close_data_updated[['Timestamp', 'Close','Y_BUY_PRED', 'Y_SELL_PRED']]
closes_5min

,Timestamp,Close,Y_BUY_PRED,Y_SELL_PRED
0,1609778700,129.413299,True,False
1,1609779000,129.148498,True,False
2,1609779300,128.820007,True,False
3,1609779600,128.580001,True,False
4,1609779900,128.351699,True,False
...,...,...,...,...
39533,1672432800,129.054992,True,False
39534,1672433100,129.380004,True,False
39535,1672433400,129.764404,True,False
39536,1672433700,129.949996,True,False


In [4]:



cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]
best_global_strategy = {'name': None, 'value': float('-inf')}

def optimize(trial):
    # Definición de los parámetros a optimizar
    stop_loss = trial.suggest_float('stop_loss', 0.00250, 0.05)
    take_profit = trial.suggest_float('take_profit', 0.00250, 0.05)
    n_shares = trial.suggest_int('n_shares', 5, 200)
    
    dnn_strat = dnn_strategy(
        df=closes_5min,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss=stop_loss,
        take_profit=take_profit
    )

    dnn_strat.run_strategy()
    
    strategy_values = {
        'dnn_strategy': dnn_strat.run_strategy()
    }
    
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_strategy['value']:
        best_global_strategy['name'] = best_strategy_name
        best_global_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [5]:
# Inicializar y ejecutar la optimización


study = optuna.create_study(direction='maximize')
study.optimize(optimize ,n_trials=10,n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_strategy['name']
best_strategy_value = best_global_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best buy overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best buy strategy parameters:", best_params)

[I 2024-04-07 16:17:33,893] A new study created in memory with name: no-name-91a80b5b-0cb6-4ae3-a59c-1503ded6f758
[I 2024-04-07 16:23:43,032] Trial 8 finished with value: 1325134.3282504492 and parameters: {'stop_loss': 0.036727117076621414, 'take_profit': 0.008745152198757941, 'n_shares': 134}. Best is trial 8 with value: 1325134.3282504492.
[I 2024-04-07 16:24:08,520] Trial 7 finished with value: 1366204.4915434397 and parameters: {'stop_loss': 0.04239952918542517, 'take_profit': 0.008412159822163333, 'n_shares': 106}. Best is trial 7 with value: 1366204.4915434397.
[I 2024-04-07 16:24:24,496] Trial 1 finished with value: 1789583.6736071254 and parameters: {'stop_loss': 0.041750629415196285, 'take_profit': 0.00931515709642577, 'n_shares': 190}. Best is trial 1 with value: 1789583.6736071254.
[I 2024-04-07 16:26:03,055] Trial 4 finished with value: 1882867.8826808468 and parameters: {'stop_loss': 0.04825176945633885, 'take_profit': 0.011159221369276027, 'n_shares': 155}. Best is trial

Best buy overall strategy: dnn_strategy with value: 20139794.66375245
Best buy strategy parameters: {'stop_loss': 0.049447774856836824, 'take_profit': 0.047451264367902334, 'n_shares': 158}
